In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle

lemmatizer = WordNetLemmatizer()

import numpy as np
from keras.models import load_model

#load model NN
model = load_model('chatbot_model.h5')
model.summary()

#Load model sklearn
# with open('SVC.pkl', 'rb') as f:
#     model = pickle.load(f)

import json
import random
intents = json.loads(open('intents.json').read())
words = json.loads(open('words_data.json').read())
classes = json.loads(open('classes_data.json').read())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               15872     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 24,583
Trainable params: 24,583
Non-trainable params: 0
_________________________________________________________________


In [2]:
print(words)
print(classes)

["'m", 'a', 'about', 'accept', 'adios', 'all', 'an', 'any', 'anyone', 'are', 'awesome', 'be', 'book', 'bracelet', 'buy', 'bye', 'camera', 'can', 'card', 'cash', 'check', 'child', 'cooker', 'cost', 'credit', 'day', 'do', 'doe', 'dress', 'for', 'frying', 'get', 'gloss', 'good', 'goodbye', 'got', 'have', 'headphone', 'heel', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'high', 'hola', 'horror', 'how', 'hya', 'i', 'introduce', 'iphone', 'is', 'kind', 'laptop', 'later', 'lip', 'lipstick', 'looking', 'lot', 'lotion', 'mastercard', 'me', 'moisturizers', 'money', 'much', 'na', 'name', 'need', 'needed', 'newest', 'next', 'nice', 'novel', 'of', 'ok', 'only', 'or', 'order', 'out', 'pair', 'pan', 'pay', 'paypal', 'please', 'pot', 'prom', 'purchase', 'recommend', 'rice', 'see', 'sell', 'shampoo', 'shirt', 'shoe', 'show', 'some', 'suggest', 'sunglass', 'take', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'too', 'toy', 'vest', 'wan', 'want', 'watch', 'what', 'who', 

In [3]:
def clean(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
    return(np.array(bag))

In [4]:
#for neural network models

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
#     print(res)

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    predict_intent = ints[0]['intent']
    # print(ints)
    res = getResponse(ints, intents)
    
    return res, ints[0]['intent']

In [4]:
#for sklearn models
def predict(sentence):
    x_vec = bow(sentence, words)
    y_pred = model.predict(np.array([x_vec]))[0]
    intent = classes[y_pred]
#     print("PREDICT: ",intent, "\n")

    return intent

def get_response(intent):
    intent_list = intents["intents"]
    for i in intent_list:
        if (i['tag'] == intent):
            result = random.choice(i['responses'])
            break
    return result

def chatbot(word):
    pred = predict(word)
    res = get_response(pred)
    return res, pred

In [15]:
# while(True):
#     w = str(input())
#     print("You: ", w)
#     res = chatbot_response(w)
#     print("Bot: ",res)
    

In [5]:
model.predict(np.array([bow("good!",words)]))

array([[9.0399169e-04, 4.0047648e-04, 9.3839180e-01, 3.9830923e-04,
        5.7327084e-02, 6.6315936e-04, 1.9151077e-03]], dtype=float32)

## Create data for Online Shopping

In [6]:
import pandas as pd
import numpy as np

data = pd.read_csv("books.csv")

arr = np.array(data)
# print(arr)

In [7]:
products = []
for i in range(len(arr)):
    pd = {
        "id": arr[i][0],
        "book": arr[i][1],
        "author": arr[i][2],
        "price": arr[i][3]
    }
    print(pd)
    products.append(pd)
products = json.dumps(products)
products

{'id': 101, 'book': 'The Lord of the Rings', 'author': 'J.R.R. Tolkien', 'price': 599999}
{'id': 102, 'book': 'The Kite Runner', 'author': 'Khaled Hosseini', 'price': 649999}
{'id': 103, 'book': 'Harry Potter and the Philosopher Stone', 'author': 'J.K. Rowling', 'price': 549999}
{'id': 104, 'book': 'Slaughterhouse-Five', 'author': 'Kurt Vonnegut', 'price': 549999}
{'id': 105, 'book': 'The Lion, the Witch, and the Wardrobe', 'author': 'C.S. Lewis', 'price': 549999}
{'id': 106, 'book': 'To Kill a Mockingbird', 'author': 'Harper Lee', 'price': 599999}
{'id': 107, 'book': 'The Book Thief', 'author': 'Markus Zusak', 'price': 499999}
{'id': 108, 'book': 'Wuthering Heights', 'author': 'Emily Bronte', 'price': 649999}
{'id': 109, 'book': 'The Catcher in the Rye', 'author': 'J.D. Salinger', 'price': 449999}
{'id': 110, 'book': 'Animal Farm', 'author': 'George Orwell', 'price': 499999}


'[{"id": 101, "book": "The Lord of the Rings", "author": "J.R.R. Tolkien", "price": 599999}, {"id": 102, "book": "The Kite Runner", "author": "Khaled Hosseini", "price": 649999}, {"id": 103, "book": "Harry Potter and the Philosopher Stone", "author": "J.K. Rowling", "price": 549999}, {"id": 104, "book": "Slaughterhouse-Five", "author": "Kurt Vonnegut", "price": 549999}, {"id": 105, "book": "The Lion, the Witch, and the Wardrobe", "author": "C.S. Lewis", "price": 549999}, {"id": 106, "book": "To Kill a Mockingbird", "author": "Harper Lee", "price": 599999}, {"id": 107, "book": "The Book Thief", "author": "Markus Zusak", "price": 499999}, {"id": 108, "book": "Wuthering Heights", "author": "Emily Bronte", "price": 649999}, {"id": 109, "book": "The Catcher in the Rye", "author": "J.D. Salinger", "price": 449999}, {"id": 110, "book": "Animal Farm", "author": "George Orwell", "price": 499999}]'

In [8]:
print(chatbot_response("good"))
#print(chatbot("good"))

('Hola! How can I help?', 'greeting')


In [9]:
#for neural network
from sklearn.metrics import accuracy_score, f1_score
testjson = json.loads(open('test_data.json').read())
testtrue = []
testpred = []
for idx in range(len(testjson)):
    testtrue.append(testjson[idx][1])
    testpred.append(chatbot_response(testjson[idx][0])[1])
print("test_true: {}".format(testtrue))
print('*'*40)
print("test_pred: {}".format(testpred))
print('*'*40)
print("Accuracy: {}".format(accuracy_score(testtrue, testpred)))
print("F1 score: {}".format(f1_score(testtrue, testpred, average = 'macro')))

test_true: ['greeting', 'greeting', 'greeting', 'greeting', 'introduce', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'payment', 'payment', 'payment', 'thanks', 'thanks', 'buyproduct', 'buyproduct', 'introduce', 'introduce', 'greeting', 'thanks', 'greeting', 'payment', 'goodbye', 'thanks', 'thanks', 'payment', 'buyproduct', 'introduce', 'greeting', 'buyproduct', 'introduce', 'payment', 'payment', 'payment', 'thanks', 'buyproduct', 'greeting']
****************************************
test_pred: ['greeting', 'greeting', 'noanswer', 'greeting', 'introduce', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'payment', 'payment', 'payment', 'greeting', 'noanswer', 'buypro

In [11]:
#for sklearn models
from sklearn.metrics import accuracy_score, f1_score
testjson = json.loads(open('test_data.json').read())
testtrue = []
testpred = []
for idx in range(len(testjson)):
    testtrue.append(testjson[idx][1])
    testpred.append(predict(testjson[idx][0]))
print("test_true: {}".format(testtrue))
print('*'*40)
print("test_pred: {}".format(testpred))
print('*'*40)
print("Accuracy: {}".format(accuracy_score(testtrue, testpred)))
print("F1 score: {}".format(f1_score(testtrue, testpred, average = 'macro')))

test_true: ['greeting', 'greeting', 'greeting', 'greeting', 'introduce', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'payment', 'payment', 'payment', 'thanks', 'thanks', 'buyproduct', 'buyproduct', 'introduce', 'introduce', 'greeting', 'thanks', 'greeting', 'payment', 'goodbye', 'thanks', 'thanks', 'payment', 'buyproduct', 'introduce', 'greeting', 'buyproduct', 'introduce', 'payment', 'payment', 'payment', 'thanks', 'buyproduct', 'greeting']
****************************************
test_pred: ['greeting', 'greeting', 'greeting', 'greeting', 'introduce', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'buyproduct', 'payment', 'payment', 'payment', 'greeting', 'greeting', 'buypro

## Build the User Interface

In [10]:
from flask import Flask, render_template, request, make_response
from flask_cors import CORS

In [ ]:
app = Flask(__name__)

CORS(app)
app.config.update(
    TEMPLATES_AUTO_RELOAD = True
)
path = "templates/main.html"

@app.route("/")
def home_page():
    return (render_template("main.html"))

@app.route('/products')
def product_page():
    return (render_template("products.html"))

@app.route("/payment")
def payment_page():
    return render_template("payment.html")

@app.route("/get_products")
def get_products():  
    res = make_response(products)
#     res.set_cookie('cross-site-cookie', samesite='None', secure=True);
    return res

@app.route("/get_bot")
def bot_response():
    text = request.args.get('msg')
    #for neural networks
    res = chatbot_response(text) 
    #for sklearn models
    #res = chatbot(text)
    
    data = json.dumps({"intent": res[1], "response": res[0]})    
    return data


if __name__ == "__main__":
    app.run(host="localhost", port=8002, debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8002/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 21:32:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:32:39] "GET /get_bot?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20hello%3C%2Fspan%3E HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:32:48] "GET /get_bot?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20who%20are%20u%20%3C%2Fspan%3E HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:32:56] "GET /get_bot?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20I%20want%20to%20buy%3C%2Fspan%3E HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:32:57] "GET /products HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:32:57] "GET /get_products HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:33:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:33:49] "GET /get_bot?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20order%3C%2Fspan%3E HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 21:33:51] "GET /payment HTTP